In [1]:
%cd ../../
%pwd

/home/jupyter-vincent/6-bap


/opt/tljh/user/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/jupyter-vincent/6-bap'

In [5]:
from chatbot_domain.benchmark import JsonQuestionParser, Benchmarker, AlwaysATestSubject, NLPTestSubject, HumanTestSubject
from chatbot_domain.chatbot import ChatBotModifier, OpenAIChatBot, DIPDomainGuard, ChatBotBuilder, RAGBuilder
from chatbot_domain.rag import VectorRetriever, FacebookDPR
from chatbot_domain.transformers import ModelBuilder
from chatbot_domain.data import loadDataSetFromDisk
from chatbot_domain import logger

logger.setLevel("DEBUG")

# chatbot = ChatBotBuilder.openAI('gpt-4-turbo').benchmarkGuard().domainGuard(DIPDomainGuard).rag(
#     RAGBuilder.fromDatasetDisk('ragData.set').vectorRetriever(FacebookDPR()), contextLength = 1024 * 8
#     ).build()

# chatbot = ChatBotBuilder.model(
#     ModelBuilder().modelName('vincentverbergt/Mistral7B-DIP').deviceMap("auto").shouldQuantize(True).build(adapted=True)
#     ).benchmarkGuard().domainGuard(DIPDomainGuard).build()

chatbot = ChatBotBuilder.model(
    ModelBuilder().modelName('vincentverbergt/Mistral7B-DIP').deviceMap("auto").shouldQuantize(True).build(adapted=True)
    ).benchmarkGuard().domainGuard(DIPDomainGuard).rag(
        RAGBuilder.fromDataset('ragData.set').vectorRetriever(FacebookDPR()), contextLength= 2048
    ).build()
parser = JsonQuestionParser('questions.json')
questions = parser.parse()

# name = input("name: ")
# benchmarker = Benchmarker(HumanTestSubject(name), questions)
benchmarker = Benchmarker(NLPTestSubject(chatbot), questions)
testResult = benchmarker.evaluate()
print(testResult)
testResult.save('./scores')

2024-04-23 11:29:32,971 - chatbot_domain - INFO - loading model
2024-04-23 11:29:32,972 - chatbot_domain - INFO - With Quantization


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.
2024-04-23 11:29:37,755 - chatbot_domain - INFO - Loading tokenizer


name:  sf


2024-04-23 11:33:50,409 - chatbot_domain - INFO - Question 1/23
2024-04-23 11:33:50,409 - chatbot_domain - DEBUG - The final prompt is Which intensity transformation makes an image strictly brighter
1. Gamma transformation.
2. Log transformation.
3. This cannot be achieved through an intensity transformation.
4. No correct answer is present.
5. This question is outside my domain of knowledge.

2024-04-23 11:33:50,410 - chatbot_domain - DEBUG - The added role was 
    You are an assistant with expertise in the Image Processing domain. Your knowledge is given through the 'CONTEXT' section.
    You will try your best to answer the questions that have anything to do with the image processing domain.
    Some topics that are covered in this domain are:
        - The perception of light and the science behind it
        - sampling and quantization of images
        - Mathematical morphology
        - Filters for noise on images
    If relevant information to the question is given in the 'CON


    # TEST RESULTS
    overal Score: 10/23
    ## breakdown:
        literal:                 6/13
        deduction:               1/4
        wrong answers:           0/3
        outside domain:          3/3        
    
